In [38]:
import urllib.request 
import nltk
#nltk.download('averaged_perceptron_tagger')


In [39]:
# List of POS tags and their meaning: https://stackoverflow.com/questions/15388831/what-are-all-possible-pos-tags-of-nltk
nltk.pos_tag(nltk.word_tokenize('time flies like an arrow'))

[('time', 'NN'),
 ('flies', 'NNS'),
 ('like', 'IN'),
 ('an', 'DT'),
 ('arrow', 'NN')]

In [43]:
# https://huggingface.co/Helsinki-NLP/opus-mt-en-he
from transformers import pipeline
pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-en-he")
print(pipe("alice in the wonderland"))


Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/299M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/781k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/869k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.05M [00:00<?, ?B/s]

[{'translation_text': 'אליס בארץ הפלאות.'}]


In [40]:
alice_http_response = urllib.request.urlopen('https://www.cs.cmu.edu/~rgs/alice-I.html')
alice_line_by_line = [line.decode('utf-8') for line in alice_http_response]


In [42]:
alice_line_by_line[0:20]

['<HTML>\n',
 '<HEAD>\n',
 "<Title>Alice's Adventures in Wonderland -- Chapter I</Title>\n",
 '</HEAD>\n',
 '<BODY>\n',
 '<H2>CHAPTER I</H2>\n',
 '                      <H2>Down the Rabbit-Hole</H2>\n',
 '\n',
 '  Alice was beginning to get very tired of sitting by her sister\n',
 'on the bank, and of having nothing to do:  once or twice she had\n',
 'peeped into the book her sister was reading, but it had no\n',
 "pictures or conversations in it, `and what is the use of a book,'\n",
 "thought Alice `without pictures or conversation?'\n",
 '<P>\n',
 '  So she was considering in her own mind (as well as she could,\n',
 'for the hot day made her feel very sleepy and stupid), whether\n',
 'the pleasure of making a daisy-chain would be worth the trouble\n',
 'of getting up and picking the daisies, when suddenly a White\n',
 'Rabbit with pink eyes ran close by her.\n',
 '<P>\n']

# Tasks

### 1. Clean the text as much as you can 
### 2. How many times 'Alice' is mentioned 
### 3. How many verbs and nouns are there in the chapter?
### 4. What is the most common verb?
### 5. Do a wordcloud for the text
### 6. Transalate the chapter to Hebrew